# Variability of the surface elevation change: 3D data

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
domains = ["greenland", "antarctica"]

## Define request

In [ ]:
collection_id = "satellite-ice-sheet-elevation-change"
request = {
    "variable": "all",
    "format": "zip",
    "climate_data_record_type": "icdr",
    "version": "3_0",
}

## Processing function

In [ ]:
def get_data(ds):
    (sec_name,) = set(ds.data_vars) & {"sec", "dhdt"}
    da = ds[sec_name]
    da.attrs["long_name"] = "Surface elevation change"

    da_err = ds[f"{sec_name}_uncert"]
    da_err.attrs["long_name"] = "Surface elevation change standard error"

    return xr.merge([da.rename("sec"), da_err.rename("sec_err"), ds["high_slope"]])

## Download and transform data

In [ ]:
datasets = {}
for domain in domains:
    print(f"{domain=}")
    ds = download.download_and_transform(collection_id, request | {"domain": domain})
    datasets[domain] = get_data(ds)

## Scatter plot

In [ ]:
fig, axs = plt.subplots(1, 2, layout="constrained")
for ax, (domain, ds) in zip(axs, datasets.items()):
    ds.plot.scatter(x="sec", y="sec_err", s=5, ax=ax, xscale="log", yscale="log")
    ax.grid()
    ax.set_title(f"{domain} ice sheet".title())
_ = fig.suptitle(
    "Scatter plot of the surface elevation change magnitude vs. the surface elevation change standard error"
)